# Exercicio 2

Implementar o ECDSA usando uma das curvas elípticas primas definidas no FIPS186-4.

In [31]:
from cryptography.hazmat.backends   import default_backend
from cryptography.hazmat.primitives import hashes, hmac

default_algorithm = hashes.SHA256

def Hash(s):
    digest = hashes.Hash(default_algorithm(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

#s é a inversa multiplicativa modular de a modulo b
#t é a inversa multiplicativa modular de b modulo a
def mod_mult_inv(a,b):
    [g,s,t] = xgcd(a,b)
    return Integer(mod(s,b))



O ECDSA foi implementado com a curva P-192. Iremos agora apresentar os parâmetros das curvas para melhor perceber esta implementação.



p - Primo que define o campo finito em que da curva

n - Ordem do gerador da curva

G - Ponto da curva

d - Chave Privada

q - Chave Publica (multiplicação de d com G)

k - Gerado aleatoriamente no processo de assinatura

(s1,s2) - Assinatura 


In [32]:
NIST = dict()
NIST['P-192'] =  {
         'p': 6277101735386680763835789423207666416083908700390324961279,
         'n': 6277101735386680763835789423176059013767194773182842284081,
         'b': '64210519e59c80e70fa7e9ab72243049feb8deecc146b9b1',
         'Gx' : '188da80eb03090f67cbf20eb43a18800f4ff0afd82ff1012',
         'Gy' : '07192b95ffc8da78631011ed6b24cdd573f977a11e794811'
        }


In [37]:
c = NIST['P-192']
p  = c['p'] 
n  = c['n']
b  = ZZ(c['b'],16)
Gx = ZZ(c['Gx'],16)
Gy = ZZ(c['Gy'],16)

#https://www.johannes-bauer.com/compsci/ecc/
#4.2

#Geração de chaves
def keyGen():
    #escolhe um inteiro aleatório 'd' tal que 0 < d < p
    #private key d
    d = ZZ.random_element(1,n-1)
    E = EllipticCurve(GF(p),[-3,b])
    G = E((Gx,Gy))
    #public key q
    q = d * G
    return (d,q,G)
    
#4.4.1
#Assinatura 
#nem o s nem o r podem ser 0
def sign(d,q,G,msg):
    s = 0
    while s==0:
        r = 0
        while r==0:
            k = ZZ.random_element(1,n-1)
            r = k*G
        s1 = mod(r[0],n)
        inv = mod_mult_inv(k,n)
        s2 = (msg+d*s1)*inv
        return (s1,s2)

def verify(s1,s2,msg,q,G):
    inv = mod_mult_inv(s2,n)
    u1 = msg*inv
    u2 = s1*inv
    u1 = ZZ(u1)
    u2 = ZZ(u2) 
    ponto = (u1*G) + (u2*q) 
    #se o módulo p[0] é igual a s1 então a assinatura é válida
    px = ponto[0]
    return mod(px,n) == s1
    
    
    

In [38]:
msg = "HELLO"
h = hash(msg)

(d,q,G) = keyGen()
(s1,s2) = sign(d,q,G,h)

ver = verify(s1,s2,h,q,G)

if ver==True:
    print "Assinatura válida"
else:
    print "Erro"


Assinatura válida
